In [398]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [399]:
train_file_data = pd.DataFrame.from_csv('labeledTrainData.tsv\labeledTrainData.tsv', sep='\t')

In [400]:
train_file_data.shape

(25000, 2)

In [401]:
train_file_data.head

<bound method NDFrame.head of          sentiment                                             review
id                                                                   
5814_8           1  With all this stuff going down at the moment w...
2381_9           1  \The Classic War of the Worlds\" by Timothy Hi...
7759_3           0  The film starts with a manager (Nicholas Bell)...
3630_4           0  It must be assumed that those who praised this...
9495_8           1  Superbly trashy and wondrously unpretentious 8...
8196_8           1  I dont know why people think this is such a ba...
7166_2           0  This movie could have been very good, but come...
10633_1          0  I watched this video at a friend's house. I'm ...
319_1            0  A friend of mine bought this film for £1, and ...
8713_10          1  <br /><br />This movie is full of references. ...
2486_3           0  What happens when an army of wetbacks, towelhe...
6811_10          1  Although I generally do not like remakes

In [402]:
train_file_data["review"][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [403]:
print(BeautifulSoup(train_file_data["review"][0], "html5lib").get_text())

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 min

In [404]:
reviews = [BeautifulSoup(_, "html5lib").get_text() for _ in train_file_data["review"].values]

In [405]:
#remove punctuations
reviews = [re.sub('[,.)(?!\\\\"]', " ", _.lower()) for _ in reviews]

In [406]:
reviews[0]

"with all this stuff going down at the moment with mj i've started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj's feeling towards the press and also the obvious message of drugs are bad m'kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for 20 mi

In [407]:
reviews = [_.split() for _ in reviews]

In [408]:
print(reviews[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', "i've", 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again', 'maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent', 'moonwalker', 'is', 'part', 'biography', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released', 'some', 'of', 'it', 'has', 'subtle', 'messages', 'about', "mj's", 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', "m'kay", 'visually', 'impressive', 'but', 'of', 'course', 'this', 'is', 'all', 'about', 'michael'

In [409]:
rem_stopword = set(stopwords.words("english"))
print(rem_stopword)

{'i', 'a', 'than', 'll', 'before', 'we', 'aren', 'no', 'had', 'm', 'hadn', 'out', 'more', 'o', 'are', 'will', 'was', 't', 'these', 'our', 'been', 'with', 'their', 'him', 'isn', 'shan', 'after', 'how', 'few', 'your', 'up', 'who', 'most', 'myself', 'now', 'above', 'any', 'once', 'she', 'other', 're', 'on', 'mightn', 'shouldn', 's', 'ourselves', 'they', 'through', 'them', 'some', 'then', 'me', 'between', 'won', 'all', 'itself', 'mustn', 'yours', 'yourselves', 'same', 've', 'but', 'just', 'should', 'only', 'which', 'again', 'in', 'too', 'where', 'wasn', 'can', 'what', 'didn', 'its', 'until', 'doesn', 'further', 'by', 'her', 'have', 'weren', 'it', 'such', 'below', 'to', 'my', 'himself', 'herself', 'does', 'hers', 'against', 'when', 'not', 'so', 'couldn', 'ma', 'be', 'this', 'if', 'd', 'ain', 'because', 'about', 'into', 'themselves', 'down', 'at', 'hasn', 'you', 'were', 'over', 'off', 'having', 'those', 'during', 'being', 'his', 'as', 'own', 'ours', 'doing', 'don', 'both', 'needn', 'or', 'di

In [410]:
processed_reviews = []
for review in reviews:
    _ = []
    for word in review:
        if word not in rem_stopword:
            _.append(word)
    processed_reviews.append(_)
processed_reviews = [" ".join(_) for _ in processed_reviews]

In [411]:
print(processed_reviews[1])

classic war worlds timothy hines entertaining film obviously goes great effort lengths faithfully recreate h g wells' classic book mr hines succeeds watched film appreciated fact standard predictable hollywood fare comes every year e g spielberg version tom cruise slightest resemblance book obviously everyone looks different things movie envision amateur critics look criticize everything others rate movie important bases like entertained people never agree critics enjoyed effort mr hines put faithful h g wells' classic novel found entertaining made easy overlook critics perceive shortcomings


In [412]:
#bag of words
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = 'english', max_features = 5000)

In [413]:
x_train = vectorizer.fit_transform(processed_reviews).toarray()

In [414]:
x_train.shape

(25000, 5000)

In [415]:
y_train = np_utils.to_categorical(train_file_data["sentiment"].values)

In [416]:
cls = RandomForestClassifier(n_estimators =100)
cls.fit(x_train, train_file_data["sentiment"].values)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [417]:
test_file_data = pd.DataFrame.from_csv(r'testData.tsv\testData.tsv', sep='\t')

In [418]:
test_data = [BeautifulSoup(_, "html5lib").get_text() for _ in test_file_data["review"].values]

In [419]:
#remove punctuations
test_data = [re.sub('[,.)(?!\\\\"]', " ", _.lower()) for _ in test_data]

In [420]:
test_data[0]

"naturally in a film who's main themes are of mortality  nostalgia  and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones  however there is a craftsmanship and completeness to the film which anyone can enjoy  the pace is steady and constant  the characters full and engaging  the relationships and interactions natural showing that you do not need floods of tears to show emotion  screams to show fear  shouting to show dispute or violence to show anger  naturally joyce's short story lends the film a ready made structure as perfect as a polished diamond  but the small changes huston makes such as the inclusion of the poem fit in neatly  it is truly a masterpiece of tact  subtlety and overwhelming beauty "

In [421]:
x_test = vectorizer.fit_transform(test_data).toarray()

In [422]:
x_test.shape

(25000, 5000)

In [423]:
result = cls.predict(x_test)

In [424]:
output = pd.DataFrame( data={'id':test_file_data.index, 'sentiment':result} )
output.to_csv("output_RandomForest.csv", index=False, quoting=2)

In [425]:
model = Sequential()
model.add(Dense(2048, input_dim=len(x_train[0]), activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 2048)              10242048  
_________________________________________________________________
dense_20 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 2050      
Total params: 12,342,274
Trainable params: 12,342,274
Non-trainable params: 0
_________________________________________________________________
None


In [426]:
model.fit(x_train, y_train, epochs=3, batch_size=50, verbose=1)

Epoch 1/3
25000/25000 [==============================] - 94s - loss: 0.3435 - acc: 0.8558    
Epoch 2/3
25000/25000 [==============================] - 101s - loss: 0.1352 - acc: 0.9481   - ETA: 8s - loss: 0. - ETA: 5s - los - ETA: 0s - loss: 0.1348 - acc: 0.94
Epoch 3/3
25000/25000 [==============================] - 107s - loss: 0.0265 - acc: 0.9922   


In [427]:
result = model.predict(x_test)
result = np.argmax(result, axis=1)
output = pd.DataFrame( data={'id':test_file_data.index, 'sentiment':result} )
output.to_csv("output_ANN.csv", index=False, quoting=2)